# Estimating Turnover in Retail Trade

### Capstone Project by Christian Furger 

## Machine Learning - Data Cleaning and Feature Encoding

The goal of this project is to estimate turnover for a given enterprise/month (e.g. for enterprise X and 2017-06) based on different features:
* Target: TOV (monthly turnover per enterprise)
* Features: VAT, employment, TOV_lag1, TOV_lag2, TOV_lag12, Activity, Size, Sample_ID, Profiling, Group, Area (Canton), Language

Because of the coronavirus pandemic, which had a huge impact on retail sales, the models will be estimated once with data including 2020, once ending december 2019.

The data will be split randomly into train/test sets. This means that the entries of the test set will be dispersed over the whole time period. The goal is not to predict turnover in the future, but estimate it for a period for which there are already answers from other enterprises available.

##### What models are you planning to use and why?
This project can be seen as multiple regression problem. It can also be seen as a classification/clustering problem, where we try to group the enterprises and use the nearest neighbor or the mean of the cluster to estimate the missing turnover data. Therefore, I plan to use the following models:
* Ridge regression
* k-NN regression
* RandomForest regression
* Clustering (k-Means)

Exept for the RandomForest approach, I will standardize the data. To find the optimal parameters, I'll use hyperparameter tuning.

For each model, there is a separate jupyter notebook. In a final notebook, all models are compared with each other.

In [1]:
# Import statements
import os
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Data Cleaning and Feature Encoding <a name="1"></a>

In [2]:
# Importing Dataset
resp_data = pd.read_csv('resp_data_prep.csv', index_col = ('OID', 'Date'), dtype = {'Stratum_Noga' : object, 'Noga_Enterprise' : object, 'Hist_Limit' : object})
resp_data.shape

(143933, 35)

In [3]:
resp_data.head()

KT   ZIP  Lang Stratum_Noga  Empl_Enterprise  Stratum_Size  \
OID     Date                                                                  
1005620 2015-01  GE  1212     2          477               32           2.0   
        2015-02  GE  1212     2          477               32           2.0   
        2015-03  GE  1212     2          477               32           2.0   
        2015-04  GE  1212     2          477               32           2.0   
        2015-05  GE  1212     2          477               32           2.0   

                 FTE_Enterprise  Profiling Sample_ID Noga_Enterprise  ...  \
OID     Date                                                          ...   
1005620 2015-01       25.969999          0        Q3            4773  ...   
        2015-02       25.969999          0        Q3            4773  ...   
        2015-03       25.969999          0        Q3            4773  ...   
        2015-04       25.969999          0        Q3            4773  ...   
        2015-05       25.969999          0        Q3            4773  ...   

                Split  SentFor  TOV  nb_modif  Year  Month  Lockdown  \
OID     Date                                                           
1005620 2015-01     0      NaN  552         1  2015      1       0.0   
        2015-02     0      NaN  554         1  2015      2       0.0   
        2015-03     0      NaN  629         1  2015      3       0.0   
        2015-04     0      NaN  546         1  2015      4       0.0   
        2015-05     0      NaN  569         1  2015      5       0.0   

                 TOV_lag1 TOV_lag2 TOV_lag12  
OID     Date                                  
1005620 2015-01     681.0    599.0     519.0  
        2015-02     552.0    681.0     523.0  
        2015-03     554.0    552.0     600.0  
        2015-04     629.0    554.0     560.0  
        2015-05     546.0    629.0     582.0  

[5 rows x 35 columns]

In [4]:
# Defining variable lists
#ID = ['OID', 'Date']
target = ['TOV']
nom_vars = ['KT', 'Lang', 'Stratum_Noga', 'Sample_ID', 'Noga_Enterprise', 'Hist_Limit', 'Channel_ID']
ord_vars = ['Stratum_Size']
disc_vars = ['Year', 'Month']
cont_vars = ['TOV_lag1', 'TOV_lag2', 'TOV_lag12', 'VAT', 'Empl_Enterprise', 'FTE_Enterprise', 'Empl_RT']
flags = ['Profiling', 'Group', 'Split', 'Sample_1', 'Sample_2', 'Lockdown']

In [5]:
# Defining datasets

# df2020: complete dataset
df2020=resp_data[target+disc_vars+nom_vars+ord_vars+cont_vars+flags]

# df2019: dataset without data from 2020
df2019=resp_data[target+disc_vars+nom_vars+ord_vars+cont_vars+flags][resp_data['Year']<2020].copy()

print('df2020: ', df2020.shape)
print('df2019: ', df2019.shape)

df2020:  (143933, 24)
df2019:  (126192, 24)


### 1.1 Defining Preprocessing Functions <a name="1.1"></a>

In [6]:
# Function for calculating percentage of missing values
def missing(df):
    missing=pd.merge(df.isnull().sum().to_frame('nb_miss').rename_axis('Var').reset_index(),df.count().to_frame('non_miss').rename_axis('Var').reset_index())
    missing["total"]=missing["non_miss"]+missing["nb_miss"]
    missing["miss_pct"]=missing["nb_miss"]/missing["total"]*100
    missing=missing[missing['miss_pct']>0]
    
    return missing.sort_values(by='miss_pct', ascending=False)

In [7]:
# Function for calculation of median and max values
cont_vars_median = []
nom_vars_max = []
ord_vars_max = []

def calc_median(df):
    
    # (A) Calculate median of continuous and discrete variables
    for c in cont_vars:
        cont_vars_median.append(np.median(df[c].dropna()))
 
    # (B) Calculate most frequent value of nominal and ordinal variables
    for n in nom_vars:
         nom_vars_max.append(df[n].value_counts().idxmax())
  
    for o in ord_vars:
         ord_vars_max.append(df[o].value_counts().idxmax()) 

    #print('cont_vars_median:', cont_vars_median)
    #print('nom_vars_max:', nom_vars_max)
    #print('ord_vars_max:', ord_vars_max)

In [8]:
# Preprocessing function
def preprocess(df):
    # Work on a copy
    df = df.copy()

    # (A) Fill missing values
            
    # Replacing missing continuous variables with median from train set
    for c in range (0,len(cont_vars)):
         df.loc[(df[cont_vars[c]].isnull()), (cont_vars[c])] =  cont_vars_median[c]
            
    # Replacing missing nominal variables with most frequent value from train set
    for n in range (0,len(nom_vars)):
         df.loc[(df[nom_vars[n]].isnull()), (nom_vars[n])] =  nom_vars_max[n]
  
    # Replacing missing ordinal variables with most frequent value from train set
    for o in range (0,len(ord_vars)):
         df.loc[(df[ord_vars[o]].isnull()), (ord_vars[o])] =  ord_vars_max[o] 

    # (B) Encoding of ordinal variables 
      
    # (C) Add new features
    
    # (D) One-hot encoding
    df = pd.get_dummies(df, columns=nom_vars+disc_vars, drop_first=True)
    
    # (E) Apply log-transform
    df[cont_vars] = np.log1p(df[cont_vars])

    # (F) Drop superfluous variables
    #df.drop(ID, axis=1, inplace=True)
        
    return df

In [9]:
# Function to check equality of number and order of columns in train and test sets
def check_cols(train,test):
    # Work on a copy
    train = train.copy()
    test = test.copy()
    print('Check: train.cols=',train.columns.shape,', test.cols=',test.columns.shape)  
    
    if train.columns.shape > test.columns.shape:
        print('Case: train > test')
        print('Missing columns:',set(list(train.columns.values.tolist()))-set(list(test.columns.values.tolist())))
        # Reindex DataFrame test with columns from train and fill new columns with 0
        train_reindexed = train
        test_reindexed = test.reindex(columns=train.columns, fill_value=0)
    else:
        print('Case: train < test')
        print('Missing columns:',set(list(test.columns.values.tolist()))-set(list(train.columns.values.tolist())))
        # Reindex DataFrame train with columns from test and fill new columns with 0
        train_reindexed = train.reindex(columns=test.columns, fill_value=0)
        test_reindexed= test
        
    print('Result: train.shape=',train_reindexed.shape,', test.shape=',test_reindexed.shape)
    return train_reindexed, test_reindexed

### 1.2 Train/Test Split <a name="1.2"></a>

In [10]:
from sklearn.model_selection import train_test_split

# Split into train/test sets
train_df2020, test_df2020 = train_test_split(
   df2020, train_size=0.7, test_size=0.3, random_state=0)

train_df2019, test_df2019 = train_test_split(
   df2019, train_size=0.7, test_size=0.3, random_state=0)

print('Train DataFrame 2020:', train_df2020.shape)
print('Test DataFrame 2020:', test_df2020.shape)
print('Train DataFrame 2019:', train_df2019.shape)
print('Test DataFrame 2019:', test_df2019.shape)

Train DataFrame 2020: (100753, 24)
Test DataFrame 2020: (43180, 24)
Train DataFrame 2019: (88334, 24)
Test DataFrame 2019: (37858, 24)


In [11]:
# Exporting testsets for predictions
test_df2020.to_csv("df2020_pred.csv", encoding='utf-8', index=True)
test_df2019.to_csv("df2019_pred.csv", encoding='utf-8', index=True)

### 1.3 Data Cleaning / Feature Encoding <a name="1.3"></a>

In [12]:
def feat_encoding(train_df,test_df):
    # Work on a copy
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    # Analyzing missing values
    print('START feature encoding')
    print('--- Missing values in train set before treatment:')
    print(train_df.isnull().sum())
    #print('---------------------------')
    print('--- Missing values in test set before treatment:')
    print(test_df.isnull().sum())   
    print('---------------------------')
    
    # Calculating median and max values from train set    
    calc_median(train_df)
    
    # Cleaning and encoding of train set
    train_df_encoded=preprocess(train_df)   
    
    # Cleaning of test set and applying median and max values from train set
    test_df_encoded=preprocess(test_df)
    
    # Checking if the number and order of columns in the test and train set are equal
    (train_df_final,test_df_final)=check_cols(train_df_encoded, test_df_encoded)
    
    print('---------------------------') 
    print('--- Missing values in train set after treatment:',train_df_final.isnull().sum().sum())
    print('--- Missing values in test set after treatment:',test_df_final.isnull().sum().sum())      
    print('END feature encoding')
    
    return train_df_final, test_df_final

In [13]:
# Data cleaning and feature encoding of df2020
(train_df2020_final,test_df2020_final)=feat_encoding(train_df2020,test_df2020)

START feature encoding
--- Missing values in train set before treatment:
TOV                  0
Year                 0
Month                0
KT                   0
Lang                 0
Stratum_Noga        47
Sample_ID            0
Noga_Enterprise     47
Hist_Limit           0
Channel_ID          37
Stratum_Size        47
TOV_lag1             0
TOV_lag2             0
TOV_lag12            0
VAT                332
Empl_Enterprise      0
FTE_Enterprise       0
Empl_RT             71
Profiling            0
Group                0
Split                0
Sample_1             0
Sample_2             0
Lockdown             0
dtype: int64
--- Missing values in test set before treatment:
TOV                  0
Year                 0
Month                0
KT                   0
Lang                 0
Stratum_Noga        28
Sample_ID            0
Noga_Enterprise     28
Hist_Limit           0
Channel_ID          12
Stratum_Size        28
TOV_lag1             0
TOV_lag2             0
TOV_lag12     

In [14]:
# Data cleaning and feature encoding of df2019
(train_df2019_final,test_df2019_final)=feat_encoding(train_df2019,test_df2019)

START feature encoding
--- Missing values in train set before treatment:
TOV                  0
Year                 0
Month                0
KT                   0
Lang                 0
Stratum_Noga        46
Sample_ID            0
Noga_Enterprise     46
Hist_Limit           0
Channel_ID          23
Stratum_Size        46
TOV_lag1             0
TOV_lag2             0
TOV_lag12            0
VAT                313
Empl_Enterprise      0
FTE_Enterprise       0
Empl_RT             80
Profiling            0
Group                0
Split                0
Sample_1             0
Sample_2             0
Lockdown             0
dtype: int64
--- Missing values in test set before treatment:
TOV                  0
Year                 0
Month                0
KT                   0
Lang                 0
Stratum_Noga        17
Sample_ID            0
Noga_Enterprise     17
Hist_Limit           0
Channel_ID           9
Stratum_Size        17
TOV_lag1             0
TOV_lag2             0
TOV_lag12     

### 1.4 Create X/y Data <a name="1.4"></a>

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE

In [16]:
# Compare models with root mean squared error (RMSE)
def RMSE(y, y_pred):
    return np.sqrt(np.mean(np.square(y-y_pred)))

In [17]:
## Complete dataset ending 2020
# Create X/y data
X2020_tr = train_df2020_final.drop(columns=['TOV']).values
y2020_tr = np.log1p(train_df2020_final.TOV).values

X2020_te = test_df2020_final.drop(columns=['TOV']).values
y2020_te = np.log1p(test_df2020_final.TOV).values

# Standardize features
scaler2020 = StandardScaler()
X2020_tr_rescaled = scaler2020.fit_transform(X2020_tr)
X2020_te_rescaled = scaler2020.transform(X2020_te)

print('X2020_tr:', X2020_tr.shape)
print('X2020_te:', X2020_te.shape)
print('X2020_tr_rescaled:', X2020_tr_rescaled.shape)
print('X2020_te_rescaled:', X2020_te_rescaled.shape)

X2020_tr: (100753, 167)
X2020_te: (43180, 167)
X2020_tr_rescaled: (100753, 167)
X2020_te_rescaled: (43180, 167)


In [18]:
## Dataset ending 2019
# Create X/y data
X2019_tr = train_df2019_final.drop(columns=['TOV']).values
y2019_tr = np.log1p(train_df2019_final.TOV).values

X2019_te = test_df2019_final.drop(columns=['TOV']).values
y2019_te = np.log1p(test_df2019_final.TOV).values

# Standardize features
scaler2019 = StandardScaler()
X2019_tr_rescaled = scaler2019.fit_transform(X2019_tr)
X2019_te_rescaled = scaler2019.transform(X2019_te)

print('X2019_tr:', X2019_tr.shape)
print('X2019_te:', X2019_te.shape)
print('X2019_tr_rescaled:', X2019_tr_rescaled.shape)
print('X2019_te_rescaled:', X2019_te_rescaled.shape)

X2019_tr: (88334, 165)
X2019_te: (37858, 165)
X2019_tr_rescaled: (88334, 165)
X2019_te_rescaled: (37858, 165)


In [19]:
# Save features into .npz files
np.savez('train_data_2020.npz', features=X2020_tr, targets=y2020_tr)
np.savez('train_data_rescaled_2020.npz', features=X2020_tr_rescaled, targets=y2020_tr)
np.savez('test_data_2020.npz', features=X2020_te, targets=y2020_te)
np.savez('test_data_rescaled_2020.npz', features=X2020_te_rescaled, targets=y2020_te)

np.savez('train_data_2019.npz', features=X2019_tr, targets=y2019_tr)
np.savez('train_data_rescaled_2019.npz', features=X2019_tr_rescaled, targets=y2019_tr)
np.savez('test_data_2019.npz', features=X2019_te, targets=y2019_te)
np.savez('test_data_rescaled_2019.npz', features=X2019_te_rescaled, targets=y2019_te)

### 1.5 Create Baseline <a name="1.5"></a>

Turnover of the same enterprise one year before is set as the baseline.

In [20]:
# Baseline 2020: turnover one year before
y2020_te_baseline = np.log1p(test_df2020_final.TOV_lag12).values

# Save baseline data into .npz file
np.savez('y2020_te_baseline.npz', targets=y2020_te_baseline)

# Calculate baseline accuracy
mae_baseline2020 = MAE(np.expm1(y2020_te), np.expm1(y2020_te_baseline))
rmse_baseline2020=RMSE(np.expm1(y2020_te), np.expm1(y2020_te_baseline))
mse_baseline2020 = MSE(y2020_te, y2020_te_baseline)

# Export baseline accuracy
baseline2020=pd.DataFrame([['Baseline', '{:.3f}'.format(mae_baseline2020), '{:.3f}'.format(rmse_baseline2020), '{:.3f}'.format(mse_baseline2020)]],
                 columns=['Model', 'MAE', 'RMSE', 'MSE'])
baseline2020.to_csv("baseline2020.csv", encoding='utf-8', index=False)
baseline2020.head()

,Model,MAE,RMSE,MSE
0,Baseline,3145.044,40103.069,15.322


In [21]:
# Baseline 2019: turnover one year before
y2019_te_baseline = np.log1p(test_df2019_final.TOV_lag12).values

# Save baseline data into .npz file
np.savez('y2019_te_baseline.npz', targets=y2019_te_baseline)

# Calculate baseline accuracy
mae_baseline2019 = MAE(np.expm1(y2019_te), np.expm1(y2019_te_baseline))
rmse_baseline2019=RMSE(np.expm1(y2019_te), np.expm1(y2019_te_baseline))
mse_baseline2019 = MSE(y2019_te, y2019_te_baseline)

# Export baseline accuracy
baseline2019=pd.DataFrame([['Baseline', '{:.3f}'.format(mae_baseline2019), '{:.3f}'.format(rmse_baseline2019), '{:.3f}'.format(mse_baseline2019)]],
                 columns=['Model', 'MAE', 'RMSE', 'MSE'])
baseline2019.to_csv("baseline2019.csv", encoding='utf-8', index=False)
baseline2019.head()

,Model,MAE,RMSE,MSE
0,Baseline,2956.581,38434.627,15.130
